In [26]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample

In [27]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [28]:
def runLDA(dfTrain):
    N, N1, N2, pi, U1, U2 = np.zeros(6)
    Y = dfTrain[20].as_matrix()
    dfTrain = dfTrain.iloc[:,0:20]
    S = np.zeros((20,20))
    S1 = np.zeros((20,20))
    S2 = np.zeros((20,20))
    N = len(dfTrain)
    N1 = np.count_nonzero(Y)
    N2 = N - N1
    pi = N1/N
    
    for i in range(0, len(dfTrain)):
        U1 = U1 + Y[i]*dfTrain.iloc[i,:].as_matrix()
        U2 = U2 + (1-Y[i])*dfTrain.iloc[i,:].as_matrix()
    U1 = U1/N1
    U2 = U2/N2
    
    for i in range(0, len(dfTrain)):
        xSubMean = np.zeros(20)
        if (Y[i]==1):
            xSubMean = np.subtract(dfTrain.iloc[i,:].as_matrix(), U1)   
            S1 = S1 + np.outer(xSubMean, xSubMean)
        elif (Y[i]==0):
            xSubMean = np.subtract(dfTrain.iloc[i,:].as_matrix(), U2)
            S2 = S2 + np.outer(xSubMean, xSubMean)
        
    S = np.add(S1, S2)
    S = np.divide(S, N)
    return (U1, U2, pi, S)

# runLDA()


In [30]:
def linearDecisionBoundary(dfTrain):
    U1, U2, pi, S = runLDA(dfTrain)
    invS = np.linalg.inv(S)
    w = np.dot(invS, np.subtract(U1, U2))
    w0 = 0
    w0 = np.divide(np.dot(U1.T, np.dot(invS, U1)), -2)
    w0 = w0 + np.divide(np.dot(U2.T, np.dot(invS, U2)), 2)
    w0 = w0 + np.log(pi/(1-pi))
    
#     print(w)
    return (w, w0)

In [31]:
def testData(dfTrain, dfTest):
    w, w0 = linearDecisionBoundary(dfTrain)
    Y = dfTest[20].as_matrix()
    dfTest = dfTest.iloc[:,0:20]
    classifier = 0
#     print(w0)
#     print(w)
    truePositive, falsePositive, falseNegative = np.zeros(3)
    for i in range(0, len(dfTest)):
        probability = np.dot(w, dfTest.iloc[i,:].as_matrix()) + w0
        if probability>=0:
            if Y[i] == 1:
                truePositive+=1
            elif Y[i] == 0:
                falsePositive+=1
                
        elif probability<0 and Y[i] == 1:
            falseNegative+=1
        
    precision = truePositive/(truePositive+falsePositive)
    recall = truePositive/(truePositive+falseNegative)
    fMeasure = 2*precision*recall/(precision+recall)
    print('Precision', precision)
    print('Recall', recall)
    print('F-Measure', fMeasure)


In [24]:
testData(readCsv('2_train.csv'), readCsv('2_test.csv'))

Precision 0.518115942029
Recall 0.462035541195
F-Measure 0.488471391973
